<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
import math
from keras.optimizers import SGD
from keras.initializers import RandomUniform
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed, Bidirectional, Embedding
from keras.layers import RepeatVector
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout, Activation
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

Using TensorFlow backend.


In [3]:
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.en', names=['src'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/train.hi', names=['dest'])
lines = pd.concat([X[:3000],Y[:3000]], axis=1)
print(len(lines))
lines = shuffle(lines)
lines.sample(10)

3000


,src,dest
1930,Opposite to the normal congenital reflex actio...,जन्मजात सामान्य प्रतिवर्ती-क्रियाओं के विपरीत ...
1242,But the number of patients coming to the psych...,पर पहले की बनिस्पत आज सायकियाट्रिस्ट या सायकोल...
1160,Now the American scientists have made that bon...,अमेरिकी वैज्ञानिकों ने अब ऐसा अस्थि-मज्जा ( हड...
2003,"STD spreads through bacteria , parasites and v...","एसटीडी बैक्टीरिया , पैरासाइटिस और वायरस से फैल..."
1091,Take water for a while two - three times in th...,पानी को थोड़ी देर हलक़ में दो-तीन बार करें ।
1263,With the decrease of brilliance in body especi...,"`` शरीर , खासकर मस्तिष्क के टिश्यूज में ओजस की..."
209,Use neem oil or mustard oil over body .,शरीर पर नीम का तेल या सरसों के तेल का उपयोग कर...
515,The tuberculosis disease spreads through air f...,तपेदिक बीमारी एक क्षय रोगी से स्वस्थ व्यक्ति क...
7,In black cataract the eye nerves dilapidates g...,काला मोतियाबिंद में नेत्र तंत्रिका धीरे -धीरे ...
1140,Uterine flow and other uterus related problems...,गर्भस्त्राव और गर्भ से संबंधित दूसरी समस्याएँ ...


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : '<SOS> '+ x + ' <EOS>')
  return data

In [0]:
lines.src = process_data(lines.src,False)
lines.dest = process_data(lines.dest,True)
# lines.dest.sample(10)

In [0]:
def get_vocab_list(data):
  vocab = []
  for line in data:
    temp = line.split()
    for item in temp:
      if item not in vocab:
        vocab.append(item)
  print(vocab)
  return vocab

In [7]:
src_vocab = get_vocab_list(lines.src)
dest_vocab = get_vocab_list(lines.dest)
print(src_vocab[:10])
print(dest_vocab[:10])
src_vocab = sorted(src_vocab)
# dest_vocab = sorted(dest_vocab)
print(src_vocab[:10])
print(dest_vocab[:10])

['by', 'blocking', 'the', 'uterine', 'artery', 'blood', 'supply', 'available', 'to', 'tumor', 'gets', 'obstructed', 'and', 'reduces', 'up', 'therefore', 'a', 'healthy', 'person', 'should', 'make', 'use', 'of', 'toilet', 'after', 'complete', 'cleaning', 'it', 'or', 'if', 'possible', 'another', 'one', 'about', 'days', 'symptoms', 'disease', 'start', 'appearing', 'in', 'that', 'for', 'study', 'diagnosis', 'thyroid', 'gland', 'also', 'there', 'is', 'less', 'need', 'using', 'radioactive', 'elements', 'due', 'ultrasound', 'with', 'which', 'both', 'treater', 'treated', 'have', 'dangers', 'radioactivity', 'several', 'symbols', 'increase', 'power', 'energy', 'taught', 'reiki', 'course', 'yellow', 'fever', 'occurs', 'an', 'arbovirus', 'family', 'flaviviridae', 'getting', 'rid', 'all', 'tiredness', 'feel', 'i', 'am', 'completely', 'away', 'from', 'negative', 'stress', 'positive', 'has', 'come', 'me', 'pay', 'full', 'attention', 'hygiene', 'get', 'check', 'done', 'regularly', 'yes', 'people', 'wea

In [8]:
print(src_vocab[:10])
print(dest_vocab[:10]) 

['a', 'aagantu', 'abdomen', 'ability', 'able', 'abnormal', 'abnormality', 'abode', 'abortion', 'abortions']
['<SOS>', 'यूटेराइन', 'आर्टरी', 'को', 'अवरुद्ध', 'करने', 'से', 'रसौली', 'मिलने', 'वाली']


In [0]:
# Max Length of source sequence
def get_max_sentence_size(data):
  max_len = 0
  for line in data:
    temp = line.split()
    max_len = max(max_len, len(temp))
  return max_len

In [10]:
src_max_len = get_max_sentence_size(lines.src)
dest_max_len = get_max_sentence_size(lines.dest)
src_max_len,dest_max_len

(84, 86)

In [11]:
src_vocab_len = len(src_vocab)
dest_vocab_len = len(dest_vocab) + 1
src_vocab_len, dest_vocab_len

(4711, 6289)

In [0]:
def get_word_to_index_dict(vocab):
  word_to_index = {}
  for i, word in enumerate(vocab):
    word_to_index[word] = i+1
  return word_to_index

In [0]:
src_word_to_index = get_word_to_index_dict(src_vocab)
dest_word_to_index = get_word_to_index_dict(dest_vocab)

In [14]:
print(src_word_to_index)
print(dest_word_to_index)

{'a': 1, 'aagantu': 2, 'abdomen': 3, 'ability': 4, 'able': 5, 'abnormal': 6, 'abnormality': 7, 'abode': 8, 'abortion': 9, 'abortions': 10, 'about': 11, 'above': 12, 'abrasion': 13, 'abscess': 14, 'absolutely': 15, 'absorbed': 16, 'absorber': 17, 'absorption': 18, 'abstain': 19, 'abstinence': 20, 'abundance': 21, 'abundant': 22, 'acacia': 23, 'accept': 24, 'acceptability': 25, 'accepted': 26, 'accessory': 27, 'accident': 28, 'accidents': 29, 'according': 30, 'accumulate': 31, 'accumulated': 32, 'accumulates': 33, 'accumulation': 34, 'acetylcholine': 35, 'ache': 36, 'achieve': 37, 'achieved': 38, 'achievement': 39, 'achievements': 40, 'acid': 41, 'acidity': 42, 'acids': 43, 'acquaintances': 44, 'acquire': 45, 'acrobats': 46, 'act': 47, 'action': 48, 'actions': 49, 'activate': 50, 'active': 51, 'activities': 52, 'activity': 53, 'actual': 54, 'actually': 55, 'acupressure': 56, 'acupuncture': 57, 'acute': 58, 'ad': 59, 'add': 60, 'added': 61, 'addiction': 62, 'adding': 63, 'addition': 64, '

In [0]:
def get_index_to_word_dict(vocab):
  index_to_word = {}
  for i, word in enumerate(vocab):
    index_to_word[i+1] = word
  return index_to_word  

In [0]:
src_index_to_word = get_index_to_word_dict(src_vocab)
dest_index_to_word = get_index_to_word_dict(dest_vocab)

In [17]:
print(src_index_to_word)
print(dest_index_to_word)

{1: 'a', 2: 'aagantu', 3: 'abdomen', 4: 'ability', 5: 'able', 6: 'abnormal', 7: 'abnormality', 8: 'abode', 9: 'abortion', 10: 'abortions', 11: 'about', 12: 'above', 13: 'abrasion', 14: 'abscess', 15: 'absolutely', 16: 'absorbed', 17: 'absorber', 18: 'absorption', 19: 'abstain', 20: 'abstinence', 21: 'abundance', 22: 'abundant', 23: 'acacia', 24: 'accept', 25: 'acceptability', 26: 'accepted', 27: 'accessory', 28: 'accident', 29: 'accidents', 30: 'according', 31: 'accumulate', 32: 'accumulated', 33: 'accumulates', 34: 'accumulation', 35: 'acetylcholine', 36: 'ache', 37: 'achieve', 38: 'achieved', 39: 'achievement', 40: 'achievements', 41: 'acid', 42: 'acidity', 43: 'acids', 44: 'acquaintances', 45: 'acquire', 46: 'acrobats', 47: 'act', 48: 'action', 49: 'actions', 50: 'activate', 51: 'active', 52: 'activities', 53: 'activity', 54: 'actual', 55: 'actually', 56: 'acupressure', 57: 'acupuncture', 58: 'acute', 59: 'ad', 60: 'add', 61: 'added', 62: 'addiction', 63: 'adding', 64: 'addition', 6

In [0]:
def reverse_train_sentences(data):
  l = data.values.tolist()
  t = []
  for item in l:
    p = item.strip().split()
    p.reverse()
    k = []
    for i in p:
      i = i.strip()
      if len(i) > 0:
        k.append(i)
    p = k
    p = " ".join(p)
    p = p.strip()
    t.append(p)
  df = pd.DataFrame({'data':t})
  return df.data  

In [19]:
# Train - Test Split
X, y = lines.src, lines.dest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train_normal = copy.deepcopy(X_train)
X_train = reverse_train_sentences(X_train)
print(X_train.shape, X_test.shape)
X_train.sample(10)

(2700,) (300,)


578     removed be can member turned drunkard the of a...
540     malfunction also may kidneys increases pressur...
1820    discomfort with micturion and tract vaginal th...
1540    district every in etc banners posters fairs he...
926     immediately gastroenterologist a to shown be s...
2216    ground the on sitting by bath taking or down s...
1340      this from benefit any get not does body the but
1237    language without imagined be society human pre...
1909    joints tiny many are there bone back of side p...
1683    treatment late than lower much also is treatme...
Name: data, dtype: object

In [0]:
def generate_batch(X = X_train, y = y_train):
  batch_size = len(X)
  encoder_input_data = np.zeros((batch_size, src_max_len),dtype='float32')
  decoder_target_data = np.zeros((batch_size, dest_max_len, dest_vocab_len),dtype='float32')
  decoder_data = np.zeros((batch_size, dest_max_len),dtype='float32')
  
  for i, (input_text, target_text) in enumerate(zip(X,y)):
      for t, word in enumerate(input_text.split()):
          encoder_input_data[i, t] = src_word_to_index[word]
      for t, word in enumerate(target_text.split()):
        if t>0:
          decoder_target_data[i, t-1, dest_word_to_index[word]] = 1
        decoder_data[i,t] = dest_word_to_index[word]
  return [encoder_input_data, decoder_data], decoder_target_data

In [0]:
latent_dim = 400

In [22]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(input_dim = src_vocab_len,  output_dim = 400, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(input_dim = dest_vocab_len, output_dim = 400, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense = Dense(dest_vocab_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
print(model.summary())

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 400)    1884400     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 400)    2515600     input_2[0][0]              

In [0]:
X, y = generate_batch(X_train, y_train)
xtest, ytest = generate_batch(X_test, y_test)

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
epochs = 50
batch_size = 128

In [25]:
model.fit(X,y,
          epochs=epochs,
          batch_size=batch_size,
          validation_data =(xtest,ytest))




Train on 2700 samples, validate on 300 samples
Epoch 1/50





2700/2700 [==============================] - 13s 5ms/step - loss: 6.4974 - acc: 0.0829 - val_loss: 5.9988 - val_acc: 0.1184
Epoch 2/50
2700/2700 [==============================] - 10s 4ms/step - loss: 5.8026 - acc: 0.1246 - val_loss: 5.8547 - val_acc: 0.1256
Epoch 3/50
2700/2700 [==============================] - 10s 4ms/step - loss: 5.5899 - acc: 0.1338 - val_loss: 5.7545 - val_acc: 0.1398
Epoch 4/50
2700/2700 [==============================] - 11s 4ms/step - loss: 5.4171 - acc: 0.1516 - val_loss: 5.6841 - val_acc: 0.1458
Epoch 5/50
2700/2700 [==============================] - 11s 4ms/step - loss: 5.2625 - acc: 0.1603 - val_loss: 5.6083 - val_acc: 0.1566
Epoch 6/50
2700/2700 [==============================] - 11s 4ms/step - loss: 5.1319 - acc: 0.1694 - val_loss: 5.5632 - val_acc: 0.1634
Epoch 7/50
2700/2700 [==============================] - 10s 4ms/step - loss: 5.0066 - acc: 0.1771 - val_loss: 5.5234 - val_acc: 0.1628


In [0]:
model.save_weights('nmt_weights.h5')

In [0]:
model.load_weights('nmt_weights.h5')

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method3
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [0]:
def decode_input_sentence(arr,inverse_mapping,skip=False):
  line = ""
  for i in range(len(arr)):
    if arr[i] == 0.0:
      break
    if  inverse_mapping[arr[i]] == "<SOS>" or inverse_mapping[arr[i]] == "<EOS>":
      continue
    line += inverse_mapping[arr[i]]+" "
    
  return line

In [0]:
def decode_model_output(model_output):

    states_value = encoder_model.predict(model_output)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = dest_word_to_index['<SOS>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = dest_index_to_word[sampled_token_index]

        if (sampled_char == '<EOS>' or len(decoded_sentence) > 50):
            break

        decoded_sentence += ' '+sampled_char
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence

In [37]:
bleu_score = 0.0
print(len(xtest))
for i in range(len(xtest[0])):
  src_input_sentence = decode_input_sentence(xtest[0][i],src_index_to_word)
  dest_input_sentence = decode_input_sentence(xtest[1][i],dest_index_to_word,True)
  output_sentence = decode_model_output(xtest[0][i])
  print("Actual Source: ",src_input_sentence)
  print("Actual Target: ",dest_input_sentence)
  print("predicted Target: ",output_sentence)
  curr_bs = calculate_bleu_score(dest_input_sentence,output_sentence)
  bleu_score += curr_bs
  print("Current BLEU Score: ",curr_bs)
  # break
print("Overall BLEU Score: ", bleu_score/len(xtest[0]))

2
Actual Source:  but in the patients of arthritis this cartilage weakens due to abrasion or else gets fully damaged because of which intense pain in the joints weakness is felt and swelling also occurs 
Actual Target:  लेकिन आर्थराइटिस के मरीजों में यह कार्टिलेज घिसघिस कर कमजोर पढ़ चुका होता है या फिर पूरी तरह से नष्ट हो जाता है जिस से जोड़ों में तेज दर्द कमजोरी महसूस होती है और सूजन भी होती है । 
predicted Target:   लेकिन उल्टियाँ होने लगती हैं ।
Current BLEU Score:  0.08116697886877472
Actual Source:  after my advice saying of meeting soon and taking of quick decision in this context he went away 
Actual Target:  मेरे परामर्श के बाद शीघ्र मिलने व इस संदर्भ में जल्दी ही कोई फैसला लेने की बात कहकर वे चले गए । 
predicted Target:   हार्मोनल परिवर्तन ।
Current BLEU Score:  0.2259005009024612
Actual Source:  this pain arises intermittently and stays for from a few minutes till several hours this is called renal chronine 
Actual Target:  गुर्दे की पथरी के ज्यादातर रोगी पीठ से पेट की तरफ आते 